# CRF based segmentation

关于CRF及CRF工具使用的示例见本项目/crf，这里不再赘述。本例演示使用CRF工具用于中文分词任务中。

## 1. 语料
不管使用HMM、CRF还是其他的模型，对于中文分词任务，训练数据是少不了的。网上有公开的北大数据集（1998年1月份的《人民日报》语料），在网上可以下载到，本例也使用该语料（本例目录/data/pfr_199801.txt）。先看一眼这个语料：

In [1]:
!head -5 ./data/pku_199801.txt

迈向/v  充满/v  希望/n  的/u  新/a  世纪/n  ——/w  一九九八年/t  新年/t  讲话/n  （/w  附/v  图片/n  １/m  张/q  ）/w  
中共中央/nt  总书记/n  、/w  国家/n  主席/n  江/nr  泽民/nr  
（/w  一九九七年/t  十二月/t  三十一日/t  ）/w  
１２月/t  ３１日/t  ，/w  中共中央/nt  总书记/n  、/w  国家/n  主席/n  江/nr  泽民/nr  发表/v  １９９８年/t  新年/t  讲话/n  《/w  迈向/v  充满/v  希望/n  的/u  新/a  世纪/n  》/w  。/w  （/w  新华社/nt  记者/n  兰/nr  红光/nr  摄/Vg  ）/w  
同胞/n  们/k  、/w  朋友/n  们/k  、/w  女士/n  们/k  、/w  先生/n  们/k  ：/w  


## 2. 语料转换

### 2.1. 随机切分训练集和测试集
将北大数据集随机切分为训练集和测试集，以确保评测的可靠性。

In [2]:
!wc -l ./data/pku_199801.txt

97584 ./data/pku_199801.txt


In [3]:
# 打乱顺序.
!cat ./data/pku_199801.txt | shuf > ./data/pku_199801.txt.shuf

# 前7万行作为训练集.
!head -70000 ./data/pku_199801.txt.shuf > ./data/pku_train.txt

# 剩余的行作为测试集.
!tail -27584 ./data/pku_199801.txt.shuf > ./data/pku_test.txt

### 2.2. 转换为序列标注的数据格式 
基于CRF方法的中文分词任务是一个序列标注，对每个单字标注一个label，然后顺序扫描label得到分词结果，常用的是4-tag（即BEMS）标注：  
* 词首，常用B表示  
* 词中，常用M表示  
* 词尾，常用E表示  
* 单子词，常用S表示  

因此对于以上的标注语料，需要转换成BEMS的序列标注形式，可以使用以下脚本完成。

In [4]:
import codecs
import sys

def tagging_format(input_file, output_file):
    input_fh = codecs.open(input_file)
    output_fh = codecs.open(output_file, 'w', encoding='utf-8')

    for line in input_fh.readlines():
        node_list = line.strip().decode('utf8').split(' ')
        for node in node_list:
            word_pos = node.split('/')
            if len(word_pos) < 2:
                continue

            word, POS = word_pos[0:2]
            if len(word) == 1:
                output_fh.write('%s\t%s\t%s\n' % (word, POS, 'S'))
            else:
                output_fh.write('%s\t%s\t%s\n' % (word[0], POS, 'B'))
                for w in word[1:len(word)-1]:
                    output_fh.write('%s\t%s\t%s\n' % (w, POS, 'M'))
                output_fh.write('%s\t%s\t%s\n' % (word[len(word)-1], POS, 'E'))
        output_fh.write('\n')

    input_fh.close()
    output_fh.close()

In [5]:
tagging_format('./data/pku_train.txt', './data/pku_train_4tag.txt')
tagging_format('./data/pku_test.txt', './data/pku_test_4tag.txt') 

得到4-tag格式的数据，展示一部分如下，第二列为当前字所在的词的词性，该列也有使用其他标记的（例如：汉字CN，数字NUM，标点符号PUNC，英文字符L等），该列作为模型特征的一部分。句子用空行分隔。

In [6]:
!head -20 ./data/pku_train_4tag.txt

中	nt	B
国	nt	M
队	nt	E
优	n	B
势	n	E
明	a	B
显	a	E
，	w	S
越	l	B
战	l	M
越	l	M
勇	l	E
，	w	S
９	m	S
号	q	S
孙	nr	S
雯	nr	S
与	c	S
１	m	B
３	m	E


## 3. 特征模板
采用常见的针对3列数据的模板文件，不在赘述具体含义，可以参考其他资料。

In [7]:
!cat template

# Unigram
U00:%x[-2,0]
U01:%x[-1,0]
U02:%x[0,0]
U03:%x[1,0]
U04:%x[2,0]
U05:%x[-1,0]/%x[0,0]
U06:%x[0,0]/%x[1,0]

U10:%x[-2,1]
U11:%x[-1,1]
U12:%x[0,1]
U13:%x[1,1]
U14:%x[2,1]
U15:%x[-2,1]/%x[-1,1]
U16:%x[-1,1]/%x[0,1]
U17:%x[0,1]/%x[1,1]
U18:%x[1,1]/%x[2,1]

U20:%x[-2,1]/%x[-1,1]/%x[0,1]
U21:%x[-1,1]/%x[0,1]/%x[1,1]
U22:%x[0,1]/%x[1,1]/%x[2,1]

# Bigram
B


## 4. 评估脚本
crfsgd工具训练时必须指定一个评估的钩子程序，默认使用的是conlleval，但是实验发现conlleval对此任务中定义的BEMS标签无效，可能需要改一下脚本，因为perl都忘了，所以没有花时间去修改，而是用python写了一个简单的评估脚本（`./seg_eval.py`），如下:

In [8]:
!cat ./seg_eval.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
 
if __name__=="__main__":
    pred_word_count = 0
    target_word_count = 0
    correct_word_count = 0
    for line in sys.stdin:
        fields = line.strip().decode('utf8').split()
        if len(fields) != 4:
            continue
    
        target, pred = fields[2:4]
        if pred in ('E', 'S'):
            pred_word_count += 1
            if target == pred:
                correct_word_count +=1
    
        if target in ('E', 'S'):
            target_word_count += 1
 
    P = correct_word_count / float(pred_word_count)
    R = correct_word_count / float(target_word_count)
    F1 = (2 * P * R) / (P + R)
    
    print('  --> Word count of predict, golden and correct : %d, %d, %d' %
            (pred_word_count, target_word_count, correct_word_count))
    print("  --> P = %f, R = %f, F1 = %f" % (P, R, F1))



## 5. 训练与评估
使用crfsgd工具进行训练，说明：  
* 参数-h 设置进行评估的步数（默认5步），会对训练集进行评估，如果同时传入测试文件，也会对训练集进行评估；
* 参数-e 设置评估命令，默认为conlleval；

In [9]:
!./bin/crfsgd -c 1.0 -f 3 -r 5 -h 1 -e "./seg_eval.py" model/seg.model template data/pku_train_4tag.txt data/pku_test_4tag.txt

Reading template file template.
  u-templates: 19  b-templates: 1
Scanning data/pku_train_4tag.txt to build dictionary.
  sentences: 69887  outputs: 4
  cutoff: 3  features: 576663  parameters: 2306664
  duration: 60.65 seconds.
Using c=1, i.e. lambda=1.43088e-05
Reading and preprocessing data/pku_train_4tag.txt.
  processed: 69887 sentences.
  duration: 77.94 seconds.
Reading and preprocessing data/pku_test_4tag.txt.
  processed: 27533 sentences.
  duration: 31.24 seconds.
[Calibrating] --  1000 samples
 initial objective=130.454
 trying eta=0.1  obj=4.014 (possible)
 trying eta=0.2  obj=8.76199 (possible)
 trying eta=0.4  obj=15.7913 (possible)
 trying eta=0.8  obj=40.1677 (possible)
 trying eta=1.6  obj=92.9449 (possible)
 trying eta=3.2  obj=125.984 (possible)
 trying eta=6.4  obj=327.972 (too large)
 trying eta=0.05  obj=2.78034 (possible)
 trying eta=0.025  obj=3.70322 (possible)
 trying eta=0.0125  obj=4.9997 (possible)
 trying eta=0.00625  obj=6.72676 (possible)
 taking eta=0.0

** 使用训练的模型 **  
对已经训练得到的模型，可以直接加载并使用，参数-t指定模型文件。下面使用上面训练的模型`model/deg.model`对测试集`data/pku_test_4tag.txt`进行评估，可以看到评估结果与上面训练阶段最后一步对测试集的评估结果是一样的。

In [10]:
!./bin/crfsgd -t model/seg.model data/pku_test_4tag.txt | ./seg_eval.py

  --> Word count of predict, golden and correct : 1571004, 1572290, 1568437
  --> P = 0.998366, R = 0.997549, F1 = 0.997958
